# Group Assignment Computer Vision

Authors:
- Robbe Neyns
- Tingyu Qu
- Emiel Vandeloo
- Po-Kai Yang

In [0]:
!pip install opencv-python
!pip install opencv-contrib-python==3.4.2.17
!pip install -q keras_vggface
!pip install -q tensorflow==1.14

This notebook explores a couple of feature representations for the task of face recognition. Two types of representations are being tested: hand-crafted features and data-driven features. Scale Invariant Feature Transform (SIFT) features are used as an example of the first, while Principal Component Analysis (PCA) and a pre-trained deep learning model are leveraged as examples of the latter. We use, evaluate and compare these methods in the context of classifying and identifying faces.

First, we build a dataset of face images. The chosen people are Ang Lee, Natalie Portman, Jack Ma and Miranda Kerr. Images of these people are extracted from the VGG-Face dataset. Next, they are cropped to only contain the face, which is done in an automated fashion by using the Haar Cascade object detection algorithm (not further discussed). 

Throughout the rest of this tutorial, these people will be identified as persons A, B, C and D, respectively. They are specially selected so persons A & C and persons B & D share some characteristics, while persons A & B and persons C & D are very dissimilar. We chose these particular people because person A & C are both men while person B and D are both women. We will use 20 training images of both person A and person B. Each person also has 10 test images.

The base images of all people are stored in the variable *images*, the faces are stored in the variable *images_face*.

In [0]:
import cv2
import numpy as np
import os
import tarfile
from google.colab.patches import cv2_imshow
from urllib import request

base_path = "/content/sample_data/CV__Group_assignment"

if not os.path.isdir(base_path):
  os.makedirs(base_path)

vgg_face_dataset_url = "http://www.robots.ox.ac.uk/~vgg/data/vgg_face/vgg_face_dataset.tar.gz"

with request.urlopen(vgg_face_dataset_url) as r, open(os.path.join(base_path, "vgg_face_dataset.tar.gz"), 'wb') as f:
  f.write(r.read())

with tarfile.open(os.path.join(base_path, "vgg_face_dataset.tar.gz")) as f:
  f.extractall(os.path.join(base_path))

trained_haarcascade_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"

with request.urlopen(trained_haarcascade_url) as r, open(os.path.join(base_path, "haarcascade_frontalface_default.xml"), 'wb') as f:
  f.write(r.read())

all_subjects = [("Ang_Lee", 32), ("Natalie_Portman", 38), ("Jack_Ma", 11), ("Miranda_Kerr", 11)]

images = []
for subject, nb_images in all_subjects:

  with open(os.path.join(base_path, "vgg_face_dataset", "files", subject + ".txt"), 'r') as f:
    lines = f.readlines()

  images_ = []
  for line in lines:
    url = line[line.find("http://"): line.find(".jpg") + 4]
    print(url)

    try:
      res = request.urlopen(url, timeout=3)
      img = np.asarray(bytearray(res.read()), dtype="uint8")
      img = cv2.imdecode(img, cv2.IMREAD_COLOR)
      h, w = img.shape[:2]
      images_.append(img)
      cv2_imshow(cv2.resize(img, (w // 5, h // 5)))

    except:
      pass

    if len(images_) == nb_images:
      images.append(images_)
      break

In [0]:
# Delete doubles and unsuitable images from the database
# Unsuitable images were identified visually
unwanted_A = [3,12]
unwanted_B = [5,8,8,10,15,17,18,22]
unwanted_C = [9]

for index in unwanted_A:
  del images[0][index]

for index in unwanted_B:
  del images[1][index]

for index in unwanted_C:
  del images[2][index]

In [0]:
faceCascade = cv2.CascadeClassifier(os.path.join(base_path, "haarcascade_frontalface_default.xml"))

images_face = []

for images_ in images:
  faces_ = []
  for img in images_:
    img_ = img.copy()
    img_gray = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        img_gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    print("Found {} face(s)!".format(len(faces)))

    if len(faces) > 1:
      faces = max(faces, key=lambda x: x[2]*x[3])
      faces = [faces]

    for (x, y, w, h) in faces:
      cv2.rectangle(img_, (x, y), (x+w, y+h), (0, 255, 0), 10)
      y_new = y - 10
      h_new = h + 10
      x_new = x - 10
      w_new = w + 10
      image_face = img[y_new:(y+h_new),x_new:(x+w_new)]
      faces_.append(image_face)
      cv2_imshow(cv2.resize(image_face, (w // 5, h // 5)))

    h, w = img_.shape[:2]
    cv2_imshow(cv2.resize(img_, (w // 5, h // 5)))

  images_face.append(faces_)

In [0]:
import matplotlib.pyplot as plt

def plot_database(images_i, title, person,columns,rows):
  w = 10
  h = 10
  fig = plt.figure(figsize=(8, 8))
  fig.suptitle(title, fontsize=16)
  for i in range(1, columns*rows +1):
      img = images_i[person][i-1]
      fig.add_subplot(rows, columns, i)
      plt.imshow(img)
  plt.show()

plot_database(images_face, 'Person A training and test set', 0, 6, 5)

In [0]:
plot_database(images_face, 'Person B training and test set', 1, 4, 5)

In [0]:
plot_database(images_face, 'Person C', 2, 2, 5)

In [0]:
plot_database(images_face, 'Person D', 3, 2, 5)

In [0]:
images[1] = images[1][:30]
images[3] = images[3][:10]

for images_ in images:
  print(len(images_))

In [0]:
for faces_ in images_face:
  print(len(faces_))

# Scale Invariant Feature Transform (SIFT)

The Scale Invariant Feature Transform (SIFT) algorithm is an example of a handcrafted feature representation. It transforms an image into a set of local feature vectors and compares images in a series of steps:

- Keypoint detection
- Keypoint suppression
- Keypoint description
- Keypoint matching

In the first phase, keypoints are detected at different scales of the image. This is done through a blob detection algorithm that is based on finding extrema of the Laplacian of the Gaussian-filtered image. These extrema are found at a certain characteristic scale $\sigma$ of the Gaussian which denotes the size of the region at which the blob is most pronounced. This detection in scale-space is the reason why the features are scale-invariant, as the same keypoint can be detected and described at a different scale after a scaling transformation. Moreover, because the Laplacian operation is rotationally invariant, so will be the feature points.

The first step results in many detected keypoints in position-scale space. In order to only retain the most prominent and robust keypoints for increased accuracy and efficiency, in the second step, we require that the contrast at the keypoint is high enough. For translation invariance, we also don't want keypoints to be on edges. However, the Laplacian response will often be high on edges. A classic approach is to look at the eigenvalues of the Hessian matrix at the position and the scale of a keypoint. It is well-known that in a corner-like structure, the eigenvalues will both be large (indicating the large change in both the x- and y-direction), whereas for an edge, only one eigenvalue is large. We only retain the keypoint when the ratio of the larger to the smaller eigenvalue is sufficiently small.

For the remaining keypoints, a (SIFT) descriptor is constructed based on the gradient information in a neighbourhood around the keypoint (usually a $16 \times 16$ patch). This patch is divided into 16 $4 \times 4$ patches, where a histogram of gradients is constructed over 8 directions. This process results in a $4 * 4 * 8 = 128$-dimensional keypoint descriptor. The orientation of the gradients is determined with respect to a dominant gradient direction in the larger patch, which makes the descriptor rotation-invariant. We can threshold the numbers for illumination invariance. The histogram (binning) results in increased robustness to noise. The descriptor is, to a certain extent, also invariant to affine transformations.

Finally, images can be compared by executing this process for both images and trying to match keypoints (descriptors) to detect whether an object (or in our case: a face) in the first image is also present in the second image. Matching can be based on a common distance measure such as the Euclidean distance.

In [0]:
trainImageA = images_face[0][:20]    # 20 training images of person A
testImageA  = images_face[0][20:]    # 10 test images of person A
trainImageB = images_face[1][:20]    # 20 training images of person B
testImageB  = images_face[1][20:]    # 10 test images of person B
testImageC  = images_face[2]         # 10 test images of person C who looks similar to Person A
testImageD  = images_face[3]         # 10 test images of person D who looks similar to Person B

We can use the built-in function SIFT_create to find the SIFT features. It accepts a couple of input parameters:

- nfeatures: The number of feature points to retain.
- nOctaveLayers: The number of layers in each octave. An octave corresponds to a certain scale. This is used for building a computationally efficient representation of the Laplacian of the Gaussian by a Difference of Gaussians.
- contrastThreshold: The contrast threshold used to filter out weak features in low-contrast regions. The larger the threshold, the less features are produced by the detector.
- edgeThreshold: The threshold used to filter out edge-like features. The larger the threshold, the more features are retained.
- sigma: The width of the Gaussian applied to the input image at the first octave.

In [0]:
def extract_sift(img, features=20, octave=10, contrast=0.03, edge=10, sigma=1):
    sift = cv2.xfeatures2d.SIFT_create(
        nfeatures=features, nOctaveLayers=octave, contrastThreshold=contrast, edgeThreshold=edge, sigma=sigma)
    kp, des = sift.detectAndCompute(img, None)
    imgFeatured = cv2.drawKeypoints(img, kp, None)
    plt.imshow(imgFeatured)
    plt.show()

    return kp, des

In order to train a binary classifier that can recognise person A (class 0) and person B (class 1), we need to construct the SIFT features for the training and the test data. The detected feature points are overlaid on the face images. We see that they indeed correspond to regions that are visually salient. In particular, the eyes, nose and mouth are often described by several feature points. However, also points on clothes, the environment and any occluding objects are sometimes incorporated, which we can expect to decrease performance. Importantly, we can assume that the feature descriptors satisfy the invariance properties discussed above, which will be important when classifying and comparing the images.

In [0]:
def getSiftFeatureAsXY(imageList):
    Xtrain = []
    for img in imageList:
      # compute SIFT keypoints & descriptors for image
      kp, des = extract_sift(img)
      Xtrain.append((des.astype(float)[:20]).ravel()) # make sure there are only 20 feature points
    return Xtrain

XtrainA = getSiftFeatureAsXY(trainImageA)
XtrainB = getSiftFeatureAsXY(trainImageB)
XtestA  = getSiftFeatureAsXY(testImageA)
XtestB  = getSiftFeatureAsXY(testImageB)
XtestC  = getSiftFeatureAsXY(testImageC)
XtestD  = getSiftFeatureAsXY(testImageD)
YtrainA = [0] * len(trainImageA)
YtrainB = [1] * len(trainImageB)
YtestA  = [0] * len(testImageA)
YtestB  = [1] * len(testImageB)
YtestC  = [0] * len(testImageC)
YtestD  = [1] * len(testImageD)

In [0]:
Xtrain  = XtrainA + XtrainB
Ytrain  = YtrainA + YtrainB
XtestAB = XtestA  + XtestB
YtestAB = YtestA  + YtestB
XtestCD = XtestC  + XtestD
YtestCD = YtestC  + YtestD

Xtrain  = np.array(Xtrain)
Ytrain  = np.array(Ytrain)
XtestAB = np.array(XtestAB)
YtestAB = np.array(YtestAB)
XtestCD = np.array(XtestCD)
YtestCD = np.array(YtestCD)

For tuning the descriptor, we apply feature matching for two images of the same person. If there are more matched features, then these descriptors are the most important ones that need to be kept. We adjust the parameters of the SIFT_create function until feature matching performs well enough for this person. The image below shows good matching correspondence. However, it is constructed for two faces that are very similar in pose and background, so we don't expect performance to be as well when testing on the whole dataset.

We found that applying blurring as a pre-processing step on the images will make some feature matching more accurate. However, in general it will reduce the number of features that are detected using the SIFT algorithm. We would like to keep the high-contrast points as salient as possible. Also, because binning over a larger patch increases robustness to noise, we chose not to apply any pre-processing for SIFT.

For detecting an object of interest in a new image, we perform feature matching with the SIFT features. Then we calculate the distance of the descriptors for the matches to assess the quality of the matches. More good matches imply that the features are more representative.

![](https://i.imgur.com/XbklZhh.png)

In [0]:
def siftFeatureMatching(img1, img2):
    
    kp1, des1 = extract_sift(img1, 500)
    kp2, des2 = extract_sift(img2, 500)

    # Create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

    # Match descriptors
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append([m])
            
    # draw image with features matching
    imgMatching = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=2)
    cv2_imshow(imgMatching)

siftFeatureMatching(trainImageA[10], trainImageA[14])

## Classification & Identification

We use the sklearn python package to build the binary classifiers and calculate the accuracy score for each classifier.

Since the SIFT algorithm has a strong invariance to scaling, rotation, brightness changes and noise, we hope to achieve good generalization to the test images of persons A & B and even to the more dissimilar images of persons C and D. The classification accuracy is 85% for persons A and B, but it is only 70% for persons C and D, using a support vector machine. We choose a support vector machine because they are generally well-suited for high-dimensional data, which is the case for 128-dimensional SIFT features and 20 feature points per image.

Although C may look similar to A and D may look similar to B, they are still different persons. It is normal that the SIFT keypoints for person A are quite different than the ones for person C (idem for persons B and D). Since we train the classifier only with images of persons A and B, the classifier does not perform well for C and D. This shows us that the generalization performance of a handcrafted feature representation like SIFT is not very good.

In [0]:
from sklearn.metrics import accuracy_score
from sklearn import svm

# training phase
clf = svm.SVC(kernel="linear")
clf.fit(Xtrain, Ytrain)

# predict labels
print("Accuracy for SVM")
predictAB = clf.predict(XtestAB)
predictCD = clf.predict(XtestCD)

# compute accuracy
print('Correct:  ', YtestAB)
print('Predicted:', predictAB)
accuracy = accuracy_score(YtestAB, predictAB) * 100
print("Accuracy for persons A & B: %.2f" % accuracy + "%\n")

print('Correct:  ', YtestCD)
print('Predicted:', predictCD)
accuracy = accuracy_score(YtestCD, predictCD) * 100
print("Accuracy for persons C & D: %.2f" % accuracy + "%")

Next, we try to perform classification with an MLP with 3 hidden layers including of 10, 8 and 3 neurons respectively. The performance gets slightly better as compared to the SVM classifier. The accuracy for persons A and B is 85%, while that for persons C and D is 75%. For two of the images for person A that is classified as person B, one of them is half-blocked by a trophy while another is half-blocked by a shadow.

In [0]:
from sklearn.neural_network import MLPClassifier

mlp_sift = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10,8,3), random_state=1)
mlp_sift.fit(Xtrain, Ytrain)

# predict labels
print("Accuracy for MLP (hidden layer: 10+8+3)")
predictAB = mlp_sift.predict(XtestAB)
predictCD = mlp_sift.predict(XtestCD)

# compute accuracy
print(np.array(predictAB))
accuracy = accuracy_score(YtestAB, predictAB) * 100
print("Accuracy for persons A & B: %.2f" % accuracy + "%\n")

print(np.array(predictCD))
accuracy = accuracy_score(YtestCD, predictCD) * 100
print("Accuracy for persons C & D: %.2f" % accuracy + "%")

Finally, we perform KNN with $K = 5$. The accuracies are exactly the same as for the SVM. To learn more about these results, we use TSNE to visualize the feature distribution. We label the test images of persons A and B with labels 2 and 3. We combine them with the training images, labelled 0 and 1. We can see in the plot that the test images of person A (green) are close to the training images of person A (blue), and the same holds for person B (red and orange). This explains the high accuracy.

For persons C and D (second plot), there is more interference: the region for persons A & C and the region for persons B & D is less clearly delineated. This explains the lower accuracy in that case.

In [0]:
from sklearn.neighbors import KNeighborsClassifier

# training phase
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(Xtrain, Ytrain)

# predict labels
print("Accuracy for KNN (K = 5)")
predictAB = neigh.predict(XtestAB)
predictCD = neigh.predict(XtestCD)

# compute accuracy
print('Correct:  ', YtestAB)
print('Predicted:', predictAB)
accuracy = accuracy_score(YtestAB, predictAB) * 100
print("Accuracy for persons A & B: %.2f" % accuracy + "%\n")

print('Correct:  ', YtestCD)
print('Predicted:', predictCD)
accuracy = accuracy_score(YtestCD, predictCD) * 100
print("Accuracy for persons C & D: %.2f" % accuracy + "%")

## SIFT features visualization

We present t-SNE for visualization. The details of t-SNE can be found in the section *Transfer Learning: t-SNE visualization*. 

Here we can see that it's easier to discriminate training data from test data of persons A and B visually. It also explains the relatively low classification accuracy of our classifiers for persons C and D.

In [0]:
from sklearn.manifold import TSNE
import seaborn as sns

# ---------------------------------
#   For testing of person A and B
# ---------------------------------


feature = np.concatenate((Xtrain, XtestAB))
feature = feature.reshape((60, 2560))

# Defining Model
tsne = TSNE(learning_rate=50, perplexity=5)

# Fitting Model
transformed = tsne.fit_transform(feature)

# Plotting 2d t-Sne
x_axis = transformed[:, 0]
y_axis = transformed[:, 1]

Ylabel = [2] * len(testImageA) + [3] * len(testImageB)
Ylabel = np.array(Ylabel)
label_tsne = np.concatenate((Ytrain, Ylabel))

palette = sns.color_palette("bright", 4)
sns.scatterplot(x_axis, y_axis, hue=label_tsne, legend='full', palette=palette)

In [0]:
# ---------------------------------
#   For testing of person C and D
# ---------------------------------

feature = np.concatenate((Xtrain, XtestCD))
feature = feature.reshape((60, 2560))

# Defining Model
tsne = TSNE(learning_rate=30, perplexity=5)

# Fitting Model
transformed = tsne.fit_transform(feature)

# Plotting 2d t-Sne
x_axis = transformed[:, 0]
y_axis = transformed[:, 1]

Ylabel = [2] * len(testImageC) + [3] * len(testImageD)
Ylabel = np.array(Ylabel)
label_tsne = np.concatenate((Ytrain, Ylabel))

palette = sns.color_palette("bright", 4)
sns.scatterplot(x_axis, y_axis, hue=label_tsne, legend='full', palette=palette)

Next, we again perform KNN ($K = 5$) with the mean-squared error between the principal component weights as a distance metric. We show the nearest neighbours of the test images of persons A and B. Here, only two of person A's images are not well clustered in feature space, where they are confused with person B. One of the images of his face is half-blocked by the trophy, while another image of his face is blocked by shadow.

For the test images of persons C and D, only two of person C's images are identified as person B. However, most of person D's images are confused with person A.

In [0]:
def visualizeKNN(k = 5):
  print('Performance on the test images of persons A and B\n')

  train_data = []
  for i in range(len(Xtrain)):
    if i < 20:
      train_data.append((Xtrain[i], 0, trainImageA[i]))
    else:
      train_data.append((Xtrain[i], 1, trainImageB[i-20]))
  
  test_data1 = []
  testImageAB = testImageA + testImageB
  for i in range(len(XtestAB)):
    test_data1.append((XtestAB[i]))
  
  test_data2 = []
  testImageCD = testImageC + testImageD
  for i in range(len(XtestCD)):
    test_data2.append((XtestCD[i]))

  labels = []
  for i in range(len(test_data1)):
    sort = sorted(train_data, key=lambda x: np.mean(np.square(np.array(x[0])-np.array(test_data1[i]))))[:k]
    sort_labels = [x[1] for x in sort]
    sort_images = [x[2] for x in sort]
    label = max(sort_labels, key=sort_labels.count)
    labels.append(label)
    print('ORIGINAL')
    cv2_imshow(cv2.resize(testImageAB[i], (128, 128)))
    print('NEIGHBOURS')
    sort_images = [cv2.resize(img, (128, 128)) for img in sort_images]
    neighbours = np.hstack((sort_images[0], sort_images[1], sort_images[2], sort_images[3], sort_images[4]))
    cv2_imshow(neighbours)

  print('\nCorrect:  ', YtestAB.tolist())
  print('Predicted:', labels)
  frac_match = np.mean([1 if YtestAB[i] == labels[i] else 0 for i in range(len(labels))])
  print('Accuracy: ', frac_match)

  print('\nPerformance on the images of persons C and D\n')

  labels = []
  for i in range(len(test_data2)):
    sort = sorted(train_data, key=lambda x: np.mean(np.square(np.array(x[0])-np.array(test_data2[i]))))[:k]
    sort_labels = [x[1] for x in sort]
    sort_images = [x[2] for x in sort]
    label = max(sort_labels, key=sort_labels.count)
    labels.append(label)
    print('ORIGINAL')
    cv2_imshow(cv2.resize(testImageCD[i], (128, 128)))
    print('NEIGHBOURS')
    sort_images = [cv2.resize(img, (128, 128)) for img in sort_images]
    neighbours = np.hstack((sort_images[0], sort_images[1], sort_images[2], sort_images[3], sort_images[4]))
    cv2_imshow(neighbours)

  print('\nCorrect:  ', YtestCD.tolist())
  print('Predicted:', labels)
  frac_match = np.mean([1 if YtestCD[i] == labels[i] else 0 for i in range(len(labels))])
  print('Accuracy: ', frac_match)

visualizeKNN(5)

# PCA

Principal Component Analysis (PCA) is a dimensionality reduction technique for high-dimensional input data. Mathematically, for linear PCA, we would like to find a matrix $D \in R^{m \times l}$ with $l < m$ such that $x \approx DD^Tx$. $D^Tx$ is the projection of $x \in R^m$ in the subspace $R^l$ such that the original point can be reconstructed in the best possible way.

Finding this matrix $D$ comes down to solving the eigenvalue problem of $XX^T$ with $X \in R^{m \times n}$ the matrix containing $n$ training data points $x_i \in R^m$. $XX^T$ has $m$ eigenvalues $\lambda_i$ and corresponding eigenvectors $u_i \in R^m$, and $D$ is constructed by retaining only the eigenvectors corresponding to the $l$ largest eigenvalues. These eigenvectors are the principal components of the training data.

Visually, the principal components are the directions in the data with the highest variance. They span an $l$-dimensional subspace onto which the $m$-dimensional datapoints can be projected with minimal loss in information. In other words, the principal components carry the highest signal of the data, while the left-out eigenvectors carry few information or are due to noise in the data. The number of principal components $l$ forms a trade-off between the amount of dimensionality reduction (efficiency of the representation) and the amount of lost information (quality of the representation). We will explore this trade-off further.

The image below shows all principal components of a two-dimensional dataset. Projecting the points onto only the most important direction clearly retains the maximal amount of information possible for a one-dimensional representation of the data.

![[Source:](https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/GaussianScatterPCA.png/512px-GaussianScatterPCA.png)](https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/GaussianScatterPCA.png/512px-GaussianScatterPCA.png)

As a pre-processing step for PCA to work, the faces need to be aligned, which means that the images of the faces are the same size and the eyes, nose, mouth, ... are more or less in the same place. The code to automate the alignment of the faces in our dataset is shown in the two cells below. 

In [0]:
import bz2

shape_predictor_dataset = "https://raw.github.com/davisking/dlib-models/master/shape_predictor_68_face_landmarks.dat.bz2"

with request.urlopen(shape_predictor_dataset) as r, open(os.path.join(base_path, "shape_predictor_dataset.dat.bz2"), 'wb') as f:
  f.write(r.read())

filepath = os.path.join(base_path, "shape_predictor_dataset.dat.bz2")
zipfile = bz2.BZ2File(filepath) # open the file
data = zipfile.read() # get the decompressed data
newfilepath = filepath[:-4] # assuming the filepath ends with .bz2
print(newfilepath)
open(newfilepath, 'wb').write(data) # write to an uncompressed file

In [0]:
import imutils
from imutils.face_utils import FaceAligner
import dlib

# By Adrian Rosebrock
# https://www.pyimagesearch.com/2017/05/22/face-alignment-with-opencv-and-python/
def align(images):
    # initialize dlib's face detector (HOG-based) and then create
    # the facial landmark predictor and the face aligner
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(os.path.join(base_path, "shape_predictor_dataset.dat"))
    fa = FaceAligner(predictor, desiredFaceWidth=256)
    aligned = []
    for image in images:
        # resize & convert to grayscale
        image = imutils.resize(image, width=800)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # detect faces in the grayscale image
        rects = detector(gray, 1)
        
        if len(rects) > 1:
          rects = [max(rects, key=lambda x: x.area())]

        # loop over the face detections
        for rect in rects:
            # extract the ROI of the *original* face, then align the face
            # using facial landmarks
            faceAligned = fa.align(image, gray, rect)
            cv2_imshow(faceAligned)
            aligned.append(faceAligned)
    return aligned

aligned_faces = []
for faces_ in images_face:
  aligned_faces.append(align(faces_))

The general method of PCA has also been applied for analyzing images of faces. Images can be transformed into a suitable vector representation by considering each colour channel of each pixel as a separate component. For example, a $100 \times 100$ image results in a vector of $100 * 100 * 3 = 30000$ components. *image_to_vec* and *vec_to_image* achieve this transformation.

Luckily, OpenCV has a function that conveniently calculates all information regarding PCA for us. Under the hood, the input images (vectors) are first transformed to have zero mean. Second, the previously discussed eigenvalue problem is solved. In this context, the returned principal components are called eigenfaces. The most important eigenfaces capture the most salient characteristics of the training faces.

Finally, we can project a new face onto the calculated principal component subspace and reconstruct the face from this lower-dimensional representation. Calculating the dot product between the face vector and the principal components gives us the projection of the face in the lower-dimensional subspace. The PCA method makes sure that this is done in an optimal way in the sense that we lose as little information as possible when performing this dimensionality reduction.

Note that the images need to be normalized to have zero mean. This is because the mean is a consequence of the translation of the data points, which cannot be accounted for when performing variance analysis and placing the principal component basis' origin. A test image also needs to be displaced according to the mean before processing, and be displaced backwards afterwards.

In [0]:
def image_to_vec(image):
    row = []
    for i in range(image.shape[0]):  # x
        for j in range(image.shape[1]):  # y
            for k in range(image.shape[2]):  # Colour
                row.append(image[i][j][k])
    return row


def vec_to_image(image_as_row):
    size = np.sqrt(image_as_row.size / 3).astype(int)
    image = np.zeros((size, size, 3), np.float32)
    for i in range(size):
        for j in range(size):
            for k in range(3):
                image[i][j][k] = image_as_row[i * size * 3 + j * 3 + k]
    return image


def pca(train_images, nr_components):
    print("Starting PCA with", str(nr_components), "components")
    train_matrix = [image_to_vec(
        cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_8UC3)
    ) for img in train_images]
    train_matrix = np.array(train_matrix)
    # Mean is automatically calculated from the data
    mean, eigen_faces = cv2.PCACompute(train_matrix, mean=None, maxComponents=nr_components)
    # print("\nEigenvalues:\n", vals)
    return mean[0], eigen_faces


def reconstruct(img, mean, eigen_faces):
    img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_8UC3)
    img_as_row = np.array(image_to_vec(img))
    rec = mean.copy()
    weights = []
    for vec in eigen_faces:
        weight = np.dot((img_as_row - mean), vec)
        weights.append(weight)
        rec += weight * vec
    rec = vec_to_image(rec)
    error = np.mean((np.square(img - rec)))
    rec = cv2.normalize(rec, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC3)
    print("\nRECONSTRUCTED")
    cv2_imshow(rec)
    print('\nError:', error)
    print('------------------------\n\n')
    return error, weights

Here, we reconstruct one of the test images of person A based on an increasing number of considered eigenfaces of the training images. We see that increasing the number of dimensions indeed reduces the error, and most significantly so with the first few eigenfaces. In this case, apparently, the third eigenface of person A captures some important characteristics of the test image, as the mean squared error drops the most and the image changes visually when we include that dimension. With a very aggressive dimensionality reduction, this may be the optimal number of components to take into account. Although the reconstruction is obviously not complete with the first 15 principal components, consider already the quality of the result with only 15 of $256 * 256 * 3 = 196608$ dimensions. Also note that we have a very challenging problem, as the training and test faces don't appear in a very uniform way that is optimal for PCA (faces turned, rotated, object partially in front of the face, high variance in the background, ...).

In [0]:
from matplotlib import pyplot as plt

def get_eigen_faces():
  train = aligned_faces[0][:20] + aligned_faces[1][:20]
  test = aligned_faces[0][20:] + aligned_faces[1][20:]
  test_image = test[1]
  print("ORIGINAL")
  cv2_imshow(test_image)

  max_components = 15
  eigen_faces = []
  errors = []
  for i in range(max_components):
      mean, vecs = pca(train, i+1)
      error, _ = reconstruct(test_image, mean, vecs)
      errors.append(error)
      eigen_faces = vecs

  plt.plot(np.linspace(1, max_components, max_components), errors)
  plt.show()
  return train, test, mean, eigen_faces

train, test, mean, eigen_faces = get_eigen_faces()

## Eigenfaces Visualization

Let's visualize the eigenfaces and observe how the first eigenfaces capture some very salient characteristics of the set of training faces, while less important eigenfaces seem to capture more detailed information about individual images and characteristics that could be attributed to noise and variance because of the heterogeneous setting in which the images were taken. Some eigenfaces clearly capture man-like characteristics (person A), other eigenfaces clearly capture woman-like characteristics (person B).

In [0]:
for ef in eigen_faces:
  cv2_imshow(cv2.normalize(vec_to_image(ef), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC3))

We now visualize the first two principal components of the test images of persons A and B. While person B is quite well clustered in the left half of the figure, there is more variance for person A. This is not necessarily a large problem when performing classification with more principal components (as we saw in the error graph above, the next principal components can still significantly improve the representation), but nevertheless it can provide an explanation when we notice that certain errors are made. As we will see, this is mostly the case in our nearest neighbours classifier in the next section.

In [0]:
weights = []
for img in test:
  _, w = reconstruct(img, mean, eigen_faces)
  weights.append(w[:2])

In [0]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

In [0]:
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

test_rgba = [np.array(cv2.cvtColor(rotate_image(img, 180), cv2.COLOR_BGR2RGBA)) for img in test]
weightsX = [w[0] for w in weights]
weightsY = [w[1] for w in weights]

p = figure(plot_width=600, plot_height=600, x_range=(-80,80), y_range=(-80,80))

p.image_rgba(test_rgba, weightsX, weightsY, [10]*len(test_rgba), [10]*len(test_rgba))

# show the results
show(p)

## Classification & Identification

Let's again try to perform classification and identification using the PCA features.

Redefine the important functions without visualizations, initialize the data and train the PCA identifier on the training images of persons A and B. *test_data1* contains the feature representations of persons A and B, *test_data2* contains the feature representations of persons C and D.

In [0]:
def pca(train_images, nr_components):
    print("Starting PCA with", str(nr_components), "components")
    train_matrix = [image_to_vec(
        cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_8UC3)
    ) for img in train_images]
    train_matrix = np.array(train_matrix)
    # Mean is automatically calculated from the data
    mean, eigen_faces = cv2.PCACompute(train_matrix, mean=None, maxComponents=nr_components)
    return mean[0], eigen_faces


def reconstruct(img, mean, eigen_faces):
    img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_8UC3)
    img_as_row = np.array(image_to_vec(img))
    weights = []
    for vec in eigen_faces:
        weight = np.dot((img_as_row - mean), vec)
        weights.append(weight)
    return weights

In [0]:
train = aligned_faces[0][:20].copy()
train.extend(aligned_faces[1][:20].copy())

test1 = aligned_faces[0][20:].copy()
test1.extend(aligned_faces[1][20:].copy())
correct1 = ([0] * len(aligned_faces[0][20:]))
correct1.extend([1] * len(aligned_faces[1][20:]))

test2 = aligned_faces[2].copy()
test2.extend(aligned_faces[3].copy())
correct2 = ([0] * len(aligned_faces[2]))
correct2.extend([1] * len(aligned_faces[3]))

mean, eigen_faces = pca(train, 15)
train_data = []
train_data_w = []
for i in range(len(train)):
  weights = reconstruct(train[i], mean, eigen_faces)
  if i < 20:
    train_data.append((weights, 0, train[i]))
    train_data_w.append(weights)
  else:
    train_data.append((weights, 1, train[i]))
    train_data_w.append(weights)

test_data1 = []
for img in test1:
  weights = reconstruct(img, mean, eigen_faces)
  test_data1.append(weights)

test_data2 = []
for img in test2:
  weights = reconstruct(img, mean, eigen_faces)
  test_data2.append(weights)

Like before, we apply an SVM classifier first. The input features in this case are the principal component weights. It can be seen that 85% of person A and B are correctly classified, while the accuracy for person C and D is 65%.

In [0]:
X = [x[0] for x in train_data]
y = [x[1] for x in train_data]

clf = svm.SVC(kernel="linear")
clf.fit(X, y)

print('Performance on the test images of persons A and B')

labels = clf.predict(test_data1)
print('Correct:  ', correct1)
print('Predicted:', labels.tolist())
frac_match = np.mean([1 if correct1[i] == labels[i] else 0 for i in range(len(labels))])
print('Accuracy: ', frac_match)

print('\nPerformance on the images of persons C and D')

labels = clf.predict(test_data2)
print('Correct:  ', correct2)
print('Predicted:', labels.tolist())
frac_match = np.mean([1 if correct2[i] == labels[i] else 0 for i in range(len(labels))])
print('Accuracy: ', frac_match)

Next, let's try to classify the test images with a 2-layer neural network.  We see that this network performs very well on the test images of persons A and B. For the images of persons C and D, which are selected to be similar to persons A and B respectively, the classifier achieves an accuracy of 75%. These results are better than those achieved with SIFT.

In [0]:
from sklearn.neural_network import MLPClassifier

X = [x[0] for x in train_data]
y = [x[1] for x in train_data]

clf = MLPClassifier(solver='lbfgs', alpha=1e-3, hidden_layer_sizes=(8,6), random_state=1)
clf.fit(X, y)

print('Performance on the test images of persons A and B')

labels = clf.predict(test_data1)
print('Correct:  ', correct1)
print('Predicted:', labels.tolist())
frac_match = np.mean([1 if correct1[i] == labels[i] else 0 for i in range(len(labels))])
print('Accuracy: ', frac_match)

print('\nPerformance on the images of persons C and D')

labels = clf.predict(test_data2)
print('Correct:  ', correct2)
print('Predicted:', labels.tolist())
frac_match = np.mean([1 if correct2[i] == labels[i] else 0 for i in range(len(labels))])
print('Accuracy: ', frac_match)

Next, we perform 3-NN with the mean-squared error between the principal component weights as a distance metric. We show the nearest neighbours of the test images of persons A and B. Apparently, person A's images are not well clustered in feature space, as he is often confused with person B. When we refer to the visualization of the first two principal components, this is indeed what we could have expected.

The pretty bad performance of PCA for nearest neighbours identification can furthermore be explained by the fact that we use an unweighted Euclidean distance when calculating the distance between the principal component weights. For PCA however, it would be useful to give more importance to the first principal components as they should be more discriminative for classification. This could be a useful extension to investigate.

In [0]:
k = 3

print('Performance on the test images of persons A and B')

labels = []
for i in range(len(test_data1)):
  sort = sorted(train_data, key=lambda x: np.mean(np.square(np.array(x[0])-np.array(test_data1[i]))))[:k]
  sort_labels = [x[1] for x in sort]
  sort_images = [x[2] for x in sort]
  label = max(sort_labels, key=sort_labels.count)
  labels.append(label)
  print('ORIGINAL')
  cv2_imshow(test1[i])
  print('NEIGHBOURS')
  neighbours = np.hstack((sort_images[0], sort_images[1], sort_images[2]))
  cv2_imshow(neighbours)

print('Correct:  ', correct1)
print('Predicted:', labels)
frac_match = np.mean([1 if correct1[i] == labels[i] else 0 for i in range(len(labels))])
print('Accuracy: ', frac_match)

print('\nPerformance on the images of persons C and D')

labels = []
for img in test_data2:
  sort = sorted(train_data, key=lambda x: np.mean(np.square(np.array(x[0])-np.array(img))))[:k]
  sort = [x[1] for x in sort]
  label = max(sort, key=sort.count)
  labels.append(label)

print('\nCorrect:  ', correct2)
print('Predicted:', labels)
frac_match = np.mean([1 if correct2[i] == labels[i] else 0 for i in range(len(labels))])
print('Accuracy: ', frac_match)

# Transfer Learning

To define transfer learning, we first introduce some notation. A domain $D$ consists of two components: a feature space $\mathcal{X}$ and a marginal probability distribution $P(X)$, where $X = \{x_{1}, ... x_{n}\} \in \mathcal{X}$. The objective function is defined as $f(·)$.

Given a source domain $D_S$ and learning task $T_S$, a target domain $D_T$ and learning task $T_T$ , transfer learning aims to help improve the learning of the target predictive function $f_{T}(·)$ in $D_T$ using the knowledge in $D_S$ and $T_S$, where $D_{S} \neq D_{T}$, or $T_{S} \neq T_{T}$. (Pan & Yang, 2010) This means that we transfer the knowledge learned in the context of one task to improve performance on a second task.

In the field of computer vision, we can leverage this technique to not need to train a network from scratch every time we encounter a new task. Instead, we reuse networks that are pre-trained on a massive dataset to learn a feature representation of our data. In this project, the VGGFace network is used to learn the feature representation of our dataset. This network is pre-trained on a dataset of 2.6M images of over 2.6K people. We will further explain the structure of the network in later sections.

---

Pan, S. J. & Yang, Q. (2010). *A Survey on Transfer Learning*. IEEE Trans. on Knowl. and Data Eng., 22, 1345--1359. DOI: 10.1109/tkde.2009.191 


In [0]:
from keras_vggface import VGGFace
import numpy as np
import os
from keras.preprocessing import image
from keras_vggface.utils import preprocess_input
from keras.utils import np_utils
from keras.layers import Input
from sklearn.utils import shuffle
from sklearn import svm
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import Image
from tensorflow.python.keras import backend as k

## Prepocessing

We start by reading all the pictures into a list, unifying the size of the pictures as $224\times224$. The VGGFace model requires the images to have the same size, and the default image size is $224\times224$. It is also possible to resize the images to other size using average pooling.

As for the preprocessing procedures for each individual image, we need to adjust the pixel values of our input images, which range from 0 to 255. Now we need to subtract the mean values provided by the VGGFace model. There are two types of input format for the VGGFace model, namely *channels_first* and *channels_last*. Using *np.expand_dims()*, we transform our input images to *channels_first* format. Then we use *preprocessing_input()* from *keras_vggface* package to subtract the means by $[93.5940, 104.7624, 129.1863]$. The values $[93.5940, 104.7624, 129.1863]$ are pre-defined according to the training data used for training the VGGFace model in the first place. We can expect slightly different values for *channels_last* input images. 

In [0]:
vgg_images = []
for faces_ in images_face:
  vgg_images_ = []
  for face in faces_:
    face = cv2.resize(face, (224, 224))
    face = image.img_to_array(face)
    face = np.expand_dims(face, axis=0)
    face = preprocess_input(face)
    vgg_images_.append(face)
  vgg_images_ = np.array(vgg_images_)
  vgg_images_ = np.rollaxis(vgg_images_, 1, 0)
  vgg_images.append(vgg_images_)
vgg_images = [x[0] for x in vgg_images]

trainset = np.concatenate((vgg_images[0][:20], vgg_images[1][:20]))
print(trainset.shape)

testset = np.concatenate((vgg_images[0][20:], vgg_images[1][20:]))
print(testset.shape)

testCD = np.concatenate((vgg_images[2][:], vgg_images[3][:]))
print(testCD.shape)

## Feature extraction using VGGFace

Here we use the pre-trained VGGFace model for feature extraction. The
VGGFace model is a deep convolutional neural network with 1 input layer,
12 convolutional layers, 5 max pooling layers, 3 fully connected (FC) 
layers and  1 softmax layer . The model was pre-trained using 2.6M
human faces.

![VGGFace structure](https://i2.wp.com/sefiks.com/wp-content/uploads/2018/08/vgg-face-model.png?ssl=1)

The input layer requires that the input images have the same size. The default size of the image is $224 \times 224$.

For all the convolutional layers, the convolutional filters have the same size of $3 \times 3$, and ReLU is used as the activation function. The output of the last convolutional layer (Conv-512) gives us a feature representation of the input images with dimension (1, 7, 7, 512). We use these features for classification and identification tasks later.

The 3 FC layers are also convolutional layers with a ReLU activation, where the size of the filters matches the size of the input data. Together with the softmax layer, these layers are used for class prediction, which we do not need in our case. We specify *include_top=False* to exclude these layers.

We extract the features for all our data and prepare the ground-truth labels.

In [0]:
vgg_features = VGGFace(include_top=False, input_shape=(224, 224, 3))

pred_feature_train = []
pred_feature_test = []
pred_feature_testCD = []

# VGG features for train set
for i in range(40):
    x = np.asarray(trainset[i])
    x = x.reshape((1, 224, 224, 3))
    pred = vgg_features.predict(x)
    pred_feature_train.append(pred)

# VGG features for test set
for i in range(20):
    x = np.asarray(testset[i])
    x = x.reshape((1, 224, 224, 3))
    pred = vgg_features.predict(x)
    pred_feature_test.append(pred)
    
# VGG features for persons C and D
for i in range(20):
    x = np.asarray(testCD[i])
    x = x.reshape((1, 224, 224, 3))
    pred = vgg_features.predict(x)
    pred_feature_testCD.append(pred)

feature_train = np.asarray(pred_feature_train)
feature_train = feature_train.reshape((40, 7*7*512))

feature_test = np.asarray(pred_feature_test)
feature_test = feature_test.reshape((20, 7*7*512))

feature_testCD = np.asarray(pred_feature_testCD)
feature_testCD = feature_testCD.reshape((20, 7*7*512))

In [0]:
# Define the number of classes
num_classes = 2
num_train = trainset.shape[0]
labels_train = np.ones((num_train,), dtype='int32')

labels_train[0:20]  = 0
labels_train[20:40] = 1

# Define the number of classes
num_test = testset.shape[0]
labels_test = np.ones((num_test,), dtype='int32')
labels_test[0:10]  = 0
labels_test[10:20] = 1

labels_testCD = np.ones((num_test,), dtype='int32')
labels_testCD[0:10]  = 0
labels_testCD[10:20] = 1

## t-SNE visualization

t-SNE is widely used for high-dimensional data visualization. In high-dimensional space, the distance between two points is mapped to a normal distribution. For those data points that are closer, the probability of them being in the same cluster will be higher.

From the original feature space, we obtain the reduced feature space by minimizing the sum of Kullback-Leibler (KL) divergence. The KL divergence measures the distance between probability distributions. The smaller it is, the closer two distributions are. Using gradient
descent, we minimize the sum of KL divergence so that we end up with the "best-fitted" reduced feature space.

We now use t-SNE for visualization. Our generated VGG features are easy to discriminate between groups, so we do not need to change most of the parameters. 

The perplexity is the most important parameter to tune. It is related to the number of nearest neighbors that is used in other manifold learning algorithms. Larger datasets usually require a larger perplexity. Here we applied four different perplexity values, namely 5, 10, 20 and 30. It is worth mentioning that the results might vary each time you train a t-SNE model.

The learning rate, which is usually within the range of [10, 1000], is set to 50. We fix the other parameters to the default.

In [0]:
perplexity_list = [5, 10, 20, 30]
for i in perplexity_list:
  # Define Model
  tsne = TSNE(learning_rate=50, perplexity=i)
  # Fit Model
  transformed = tsne.fit_transform(feature_train)
  # Plot 2d t-Sne
  x_axis = transformed[:, 0]
  y_axis = transformed[:, 1]

  label_tsne = labels_train
  
  plt.figure()
  palette = sns.color_palette("bright", 2)
  sns.scatterplot(x_axis, y_axis, hue=label_tsne, legend='full', palette=palette)

Next, we visualize the feature representations of the 4 persons A, B, C and D using t-SNE. Based on the results of the previous section, we set the perplexity to 10. It can be seen that we obtain two clusters of features, one for persons A and C and one for persons B and D. No significant difference between persons A and B and between persons C and D can be observed. We can expect our classifiers to do a great job with very good generalization in the next section.

In [0]:
feature_all = np.concatenate((feature_train, feature_test))
feature_all = feature_all.reshape((60, 7*7*512))

# Defining Model
tsne = TSNE(learning_rate=50, perplexity=10)

# Fitting Model
transformed = tsne.fit_transform(feature_all)

# Plotting 2d t-Sne
x_axis = transformed[:, 0]
y_axis = transformed[:, 1]

yLabel = [2] * len(testImageC) + [3] * len(testImageD)
yLabel = np.array(yLabel)
label_tsne2 = np.concatenate((labels_train, yLabel))

palette = sns.color_palette("bright", 4)
sns.scatterplot(x_axis, y_axis, hue=label_tsne2, legend='full', palette=palette)

## Classification & Identification

Here we use a SVM with a linear kernel as our classifier. It can be seen that all the test images of all persons are correcly classified.

In [0]:
svm_clf = svm.SVC(kernel='linear')
svm_clf.fit(feature_train, labels_train)

print('Performance on test images of persons A and B')
pred_label_svm = svm_clf.predict(feature_test).reshape((20, 1))
print(confusion_matrix(pred_label_svm, labels_test))

print('\nPerformance on test images of persons C and D')
pred_labelCD_svm = svm_clf.predict(feature_testCD).reshape((20, 1))
print(confusion_matrix(pred_labelCD_svm, labels_testCD))

We achieve the same performance with a 2-layer neural network 

In [0]:
mlp_clf2 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(12,3), random_state=1)
mlp_clf2.fit(feature_train, labels_train)
pred_label_mlp2 = mlp_clf2.predict(feature_test)

print('Performance on test images of persons A and B')
pred_label_mlp2 = pred_label_mlp2.reshape((20,1))
print(confusion_matrix(pred_label_mlp2, labels_test))

print('\nPerformance on test images of persons C and D')
pred_labelCD_mlp2 = mlp_clf2.predict(feature_testCD).reshape((20, 1))
print(confusion_matrix(pred_labelCD_mlp2, labels_testCD))

Finally, KNN with $K = 5$ is applied and all the test images of persons A and B are identified correctly. Moreover, we generalize very well to the test images of persons C and D.

In [0]:
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(feature_train, labels_train)

print('Performance on test images of persons A and B')
pred_label_knn = knn_clf.predict(feature_test).reshape((20, 1))
print(confusion_matrix(pred_label_knn, labels_test))

print('\nPerformance on test images of persons C and D')
pred_labelCD_knn = knn_clf.predict(feature_testCD).reshape((20, 1))
print(confusion_matrix(pred_labelCD_knn, labels_testCD))

# Impress the TA's

## Recognizing a person in a crowd using a sliding window

A frequently discussed topic in computer vision is how to find whether an object of interest is present in an image. The most straightforward approach to solving this is using a sliding window. Sliding windows play an integral role in object classification, as they allow us to localize precisely where in an image an object resides. We simply slide a window over an image and we each time try to identify whether this window contains the object or person we are looking for. Here we want to find Natalie Portman (person B) in a crowd of people.

First, we load the image with many people in the background and the target person, Natalie Portman, in the foreground.

In [0]:
import imutils
from PIL import Image
import requests
from io import BytesIO

In [0]:
# load the image
url = "https://images.thestar.com/0nxeh0-5bN08trlMT-wG8OYdCoU=/1200x800/smart/filters:cb(2700061000)/https://www.thestar.com/content/dam/thestar/entertainment/movies/2015/08/10/natalie-portman-in-spotlight-for-tiff-fundraiser/portman.jpg"

response = requests.get(url)
img = Image.open(BytesIO(response.content))
img = np.array(img) 
# Convert RGB to BGR 
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

img = cv2.resize(img, (640, 640))
cv2_imshow(img)
(winW, winH) = (320, 320)

Next, we build the image pyramid function. An image pyramid is a multi-scale representation of an image. Via the image pyramid, we are able to find the target object at different scales in the image. We also need to set the minimum size of the image.


In [0]:
def pyramid(image, scale=1.5, minSize=(30, 30)):
	# yield the original image
	yield image
	# keep looping over the pyramid
	while True:
		# compute the new dimensions of the image and resize it
		w = int(image.shape[1] / scale)
		image = imutils.resize(image, width=w)
		# if the resized image does not meet the supplied minimum
		# size, then stop constructing the pyramid
		if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
			break
		# yield the next image in the pyramid
		yield image

    
def sliding_window(image, stepSize, windowSize):
	# slide a window across the image
	for y in range(0, image.shape[0], stepSize):
		for x in range(0, image.shape[1], stepSize):
			# yield the current window
			yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

### Sliding window with PCA features

For the classification of Natalie Portman inside the window, we first use a two-layer neural network with PCA as features, since we obtained a good accuracy of 95% with it. We redefine the classification function here in a function since we will need to call it every time in the for loop while sliding the window. Since we will need to use the mean value and eigen faces fetched from the training images during PCA feature extraction to build the features in the test window image, we also return these.


In [0]:
def classification():
  
  # Training data

  train = aligned_faces[0][:20].copy()
  train.extend(aligned_faces[1][:20].copy())

  # PCA

  mean, eigen_faces = pca(train, 15)
  train_data = []
  for i in range(len(train)):
    weights = reconstruct(train[i], mean, eigen_faces)
    if i < 20:
      train_data.append((weights, 0))
    else:
      train_data.append((weights, 1))
  
  # Train classifier

  X = [x[0] for x in train_data]
  y = [x[1] for x in train_data]

  clf = MLPClassifier(solver='lbfgs', alpha=1e-3, hidden_layer_sizes=(8,6), random_state=1)
  clf.fit(X, y)

  return clf, mean, eigen_faces

In [0]:
# Construct the model
mlp, mean_C, eigen_faces_C = classification()


After applying classification to every window, we keep those windows that are classified as containing person B (label = 1), which is Natalie Portman. By observing the final result, every printed image includes Natalie Portman; this means that the sliding window works well for detecting the target person in a group of various people.

In [0]:
(winW, winH) = (256, 256)  # size for the sliding window
image_pos1 = []

# loop over the image pyramid
for resized in pyramid(img, scale=1.5):
  # loop over the sliding window for each layer of the pyramid
  for (x, y, window) in sliding_window(resized, stepSize=20, windowSize=(winW, winH)):
    # if the window does not meet our desired window size, ignore it
    if window.shape[0] != winH or window.shape[1] != winW:
      continue
    window = cv2.resize(window, (256, 256))
    
    weights = reconstruct(window, mean_C, eigen_faces_C)
    
    classif = mlp.predict([weights])
    if classif == 1:
      image_pos1.append(window)
  
for frame in image_pos1:
  cv2_imshow(frame)

### Sliding window with VGG features

Similarly, we apply the same procedure for features generated using VGGFace. Since we obtain perfect classification results from the KNN classifier, we apply the KNN classifier (with $K = 5$) again.

In [0]:
# Consruct the model
knn_clf1 = KNeighborsClassifier(n_neighbors=5)
knn_clf1.fit(feature_train, labels_train)

In [0]:
(winW, winH) = (256, 256)  # size for the sliding window
image_pos2 = []

# loop over the image pyramid
for resized in pyramid(img, scale=1.5):
  # loop over the sliding window for each layer of the pyramid
  for (x, y, window) in sliding_window(resized, stepSize=20, windowSize=(winW, winH)):
    # if the window does not meet our desired window size, ignore it
    if window.shape[0] != winH or window.shape[1] != winW:
      continue
    window = cv2.resize(window, (224, 224))

    x = image.img_to_array(window)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    x = x.reshape((1, 224, 224,3))

    vggf =  vgg_features.predict(x)
    vggf = vggf.reshape((1, 7*7*512))
    classif = knn_clf1.predict(vggf)
    if classif == 1:
      image_pos2.append(window)
  
for frame in image_pos2:
  cv2_imshow(frame)

# Discussion

Troughout this assignment we tested several different feature representation methods for the task of face classification. Each feature representation has its strenghts and weaknesses. However, by using the representations as input for different classifiers, it became apparent that some give better overall results than others.

**Overview of results**


|                  | **SIFT** | **PCA**  | **Transfer learning** |
| ---------------- | -------- | -------- | --------------------- |
| **Person A & B** | 85%      | 85%      | 100%                  |
| **Person C & D** | 70%      | 65%      | 100%                  |

Table 1: Accuracy obtained by the linear support vector machine classifier on the test set for each feature representation.



|                  | **SIFT** | **PCA**  | **Transfer learning** |
| ---------------- | -------- | -------- | --------------------- |
| **Person A & B** | 85%      | 100%     | 100%                  |
| **Person C & D** | 75%      | 75%      | 100%                  |

Table 2: Accuracy obtained by the multi-layer perceptron classifier on the test set for each feature representation.

|                  | **SIFT** | **PCA**  | **Transfer learning** |
| ---------------- | -------- | -------- | --------------------- |
| **Person A & B** | 85%      | 75%      | 100%                  |
| **Person C & D** | 70%      | 60%      | 100%                  |

Table 3: Accuracy obtained by the K nearest neighbours classifier on the test set for each feature representation. 



**SIFT.** 

The advantage of SIFT features, and the reason for which they were designed,  is their invariance properties with respect to rotation, scaling and affine transformations. While the images of the same person in our dataset are quite heterogeneous, there are still some notable differences in appearance, illumination, background and pose. The strength of the invariance properties of SIFT features is shown in the fact that we could achieve very good classification and identification performance on the test images of persons A and B. However, SIFT does struggle with occlusions and large shadows, this can be seen in the test image of person A where he is holding an object in front of his face, this image is always wrongly classified. 

It is also not necessary to have a large dataset for feature extraction; SIFT is a general and readily applicable technique that can immediately be used on individual images.

This final point is also a disadvantage. Because SIFT is not data-driven, the feature space that it produces cannot be optimized for a specific task or a specific set of instances. This can be seen in the significatntly worse classification accuracy for persons C and D. The features of these persons are not constructed in the context of the appearances of persons A and B, making it difficult to immediately match them and assign person C to A and D to B. The features are constructed in isolation and appear to differ significantly. Also, SIFT is quite computationally expensive.

**PCA.** 

PCA is very efficient in the sense that we can achieve a very strong dimensionality reduction, while keeping the discriminative power to classify images reasonably well. We also have flexibility in the number of principal components to retain. It slightly improves classification performance over the SIFT features, and importantly, it does so with a large reduction in the number of features.

The disadvantage is that for PCA to really work well, training and test images need to have a very coherent appearance. We perform an alignment step to reduce the variance between the images, but differences in pose, clothes, background... still significantly decrease performance. Also, the training set we use does not allow us to push the limits of the performance we can achieve, as we would need more training images to be able to increase the useful number of principal components. Using a large dataset of specifically tailored images for PCA would allow us to really assess the quality of this technique, which could be an extension when we had more time. While this reduces the robustness and usability of this technique, it is still interesting that we can achieve quite good results on a difficult task for PCA with very few features.

**Transfer learning.** 

It is clear that transfer learning is by far the best approach we have used in this project. It performs perfectly in all ways. This is because we can use an extensive neural network that has the possibility to very flexibly learn all kinds of latent characteristics that are present in the images. This results in a projection of the images to a very discriminative, optimized feature representation for the task of person identification. A disadvantage is that we need an enormous amount of data to train the network, but transfer learning alleviates this issue by leveraging a pretrained network where this has already been done.

**General conclusions.** 

As a general assessment of the performance of the difference methods, the most significant difference between handcrafted and data-driven features is that handcrafted features are constructed in isolation and only afterwards used to distinguish between the images. SIFT features are generally designed to be as robust and discriminative as possible, but they aren’t explicitly optimized for a particular set of instances or for a particular task during construction. This makes it difficult for them to generalize well, as the projection of images to feature space is not learned in the context of relations that exist between images.

However, the data-driven methods perform projections of images to a feature space that is explicitly learned to be as discriminative as possible for a particular set of instances. This means that data-driven methods are more performant because they perform the tasks of classification and identification in a space that is flexibly learned to be optimal in some way to distinguish between a particular set of images.

Nevertheless, transfer learning outperforms PCA. Transfer learning performs better than PCA because PCA learns a feature space that is optimized with respect to the statistical measure of variance, whereas transfer learning projects images to a space that is optimized with respect to arbitrary latent characteristics of the images under consideration. This makes transfer learning more flexible than PCA because it is not restricted to only dealing with variance and it has a lot of freedom to learn something optimal for the task at hand. This is a major advantage of deep learning and it is the reason why it has been so popular in recent years.

**Additional remarks and possible improvements**

The previous section provides a detailed discussion on each of the feature representations and their relative strengths and weaknesses. In this final section we go a bit further and try to explore some ways in which our feature representation and therefore the classifier could be further improved. We also look at some possible pitfalls when using this system as a security face identification system.
 
Due to the limited time-frame in which this assignment had to be completed, we did not get to implement all our ideas to improve the system. To improve the performance of the SIFT feature algorithm it would be interesting to implement SURF (Bay et al., 2006) or ORB (Rublee et al., 2011). To add color invariance to the SIFT features we could also implement CSIFT (Abdel-Hakim & Farag, 2006).

Various improvements are still possible/necessary to our most performant feature representation: the transfer learning features, especially to make it function properly as a company security system. At the moment, the dataset is very small and only contains two people. Therefore, the network will predict either person A or person B when presented with a picture. In a real-world setting this is undesirable, we need a third class with faces and even objects that are ‘not known’. An efficient way of training this third class is to use our moving window implementation. By applying our moving window algorithm to different pictures that are very crowded, we will have at once a large number of false positives on which the network can be retrained. 

Another necessary step is to make the network more robust to adversarial attacks. One way of doing this has already been mentioned, namely creating a larger dataset. However, it has been shown that this alone is not enough. The reason for this is that, even with a large dataset, the machine learning models are only trained on a small subset of the whole feature space. The high-dimensional feature space, especially in image analysis, is so sparse that most of the training data is located only in a small region; this region is referred to as the manifold. Therefore, the network can be fooled by creating images that fall outside of the manifold. This can be done by something as simple as wearing glasses with a specific pattern (Sharif et al., 2016).  Knowing this, specific datasets have to be used to test the robustness of the network and improve it. An example of such a dataset is Damagenet which was created for the ImageNet dataset and able to fool many models trained on the dataset (Chen et al., 2019)

**Bibliography**

Abdel-Hakim, Alaa & Farag, Aly. (2006). CSIFT: A SIFT descriptor with color invariant characteristics. Proceedings of the IEEE Computer Society Conference on Computer Vision and Pattern Recognition. 2. 1978 - 1983. 10.1109/CVPR.2006.95. 

Bay, H., Tuytelaars, T., & Van Gool, L. (2006, May). Surf: Speeded up robust features. In 
European conference on computer vision (pp. 404-417). Springer, Berlin, Heidelberg.

Chen, S., Huang, X., He, Z., & Sun, C. (2019). DAmageNet: A Universal Adversarial Dataset
. arXiv:1912.07160

Rublee, E., Rabaud, V., Konolige, K., & Bradski, G. (2011, November). ORB: An efficient alternative to SIFT or SURF. In 2011 International conference on computer vision (pp. 2564-2571). 

Sharif, M., Bhagavatula, S., Bauer, L., & Reiter, M. K. (2016, October). Accessorize to a crime: Real and stealthy attacks on state-of-the-art face recognition. In Proceedings of the 2016 acm sigsac conference on computer and communications security (pp. 1528-1540).

